# 1. Set up your environment

In [ ]:
!pip install langchain langchain-community chromadb transformers sentence-transformers bitsandbytes beautifulsoup4 unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Crawl & load the website content

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "https://inceptionbd.com/",
    "https://inceptionbd.com/classes?sort=newest/",
    "https://inceptionbd.com/instructor-finder/",
    "https://inceptionbd.com/products/",
    "https://inceptionbd.com/forums/",

]

loader = UnstructuredURLLoader(urls=urls)
raw_docs = loader.load()

In [ ]:
raw_docs

[Document(metadata={'source': 'https://inceptionbd.com/'}, page_content="Your cart is empty\n\nEmpty notifications\n\nLogin Register\n\nLet's inception for the future...\n\nTake your career to the next level with industry ready programs,An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.\n\n\n\nSkillful Instructors\n\nStart learning from experienced instructors.\n\n\n\n659\n\nHappy Students\n\nEnrolled in our courses and improved their skills.\n\n\n\nLive Classes\n\nImprove your skills using live knowledge flow.\n\n\n\nVideo Courses\n\nLearn without any geographical & time limitations.\n\nFeatured Courses\n\n#Browse featured courses and become skillful\n\nMaster Circuit Simulation with Proteus\n\n\n\nDabasish Kumar Biswas\n\nJoin Inception BD's exclusive live class on Proteus Software! Learn circuit design, simulation, PCB layout, microcontroller programming, IoT applications, and more. Perfect

# 3. Split documents into manageable chunks

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(raw_docs)

# 4. Embed the chunks & build your ChromaDB vector store

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Use a lightweight HF embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Create (or connect to) a local Chroma collection
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="inception_website"
)

# 5. Load Llama 2 via HuggingFace and wrap as a LangChain LLM

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `MY_NEW_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "daryl149/llama-2-7b-chat-hf"

# 1. Tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_auth_token=True,)



model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )



# 2. Wrap in a text-generation pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


# 3. LangChain LLM
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-9-471044274>:43: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

<ipython-input-10-1177971522>:1: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a concise summary of the Book Harry Potter")


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone by J.K. Rowling.\nHarry Potter and the Philosopher's Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter who discovers that he is famous in the wizarding world. He attends Hogwarts School of Witchcraft and Wizardry, where he makes friends with Ron Weasley and Hermione Granger. Together, they become entangled in a mystery surrounding the powerful Sorcerer's Stone, which is being kept at Hogwarts. The trio must uncover the truth about the stone and stop the evil wizard, Lord Voldemort, from obtaining it.\nThe book introduces the magical world of Harry Potter, including the three main characters and their adventures at Hogwarts. The story explores themes of friendship, courage, and the battle between good and evil. The book was first published in 1997 and has since become a bestseller and a beloved classic in children's literature."

# 6. Assemble the RetrievalQA chain

In [ ]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
query = "Tell me about Inception BD?"
answer = qa_chain.run(query)
print(answer)

<ipython-input-12-3384863621>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Student, Magura Polytechnic Institute

Highly recommended for tech enthusiasts! Inception BD provides hands-on experience in robotics and cutting-edge technology.

MD Titas Sheike

MD Titas Sheike

Student, Jhenaidah Polytechnic Institute

Inception BD bridges the gap between theoretical knowledge and practical skills in robotics and Programming.

MD Likhon Mia

MD Likhon Mia

Student, Jhenaidah Polytechnic Institute

Best place to learn advanced programming and gain hands-on experience with modern technology. Thanks Inception BD!

Abdur Rahman

Abdur Rahman

Student, Deshari Institute Of Science And Engineering.

Outstanding mentorship and resources for tech enthusiasts! Inception BD ensures a practical learning experience.

Win Club Points

medal

You earned 50 points! for completing the course...

Win Club Points

Program

In [ ]:
query = "How to purchase a course?"
answer = qa_chain.run(query)
print(answer)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Your cart is empty

Empty notifications

Login Register

Let's inception for the future...

Take your career to the next level with industry ready programs,An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.

Skillful Instructors

Start learning from experienced instructors.

659

Happy Students

Enrolled in our courses and improved their skills.

Live Classes

Improve your skills using live knowledge flow.

Video Courses

Learn without any geographical & time limitations.

Featured Courses

#Browse featured courses and become skillful

Master Circuit Simulation with Proteus

Dabasish Kumar Biswas

Join Inception BD's exclusive live class on Proteus Software! Learn circuit design, simulation, PCB layout, microcontroller programming, 

In [ ]:
query = "I want to enroll Gen AI Course. Give me the full guide ilne simply"
answer = qa_chain.run(query)
print(answer)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

4.50

30:00 Hours

14 Feb 2025

Free

Master Generative AI in Bangla

Boktiar Ahmed Bappy

Unlock the power of Generative AI with this comprehensive Bangla course! Dive into AI model training, deployment, and optimization while exploring popular tools like LLMs and GANs. Gain hands-on experience by building real-world projects, including chatbots, image generators, and creative content applications. Whether you're a beginner or an AI enthusiast, this course equips you with in-demand skills to excel in the evolving world of AI. Master the art of creating AI-driven text, images, and more, and become proficient in one of the most exciting fields of technology today!

4.95

100:00 Hours

16 Jun 2025

Free

Newest Courses

#Recently published courses

View All

Bestselling Courses

#Learn from bestselling courses

View All

Instr

In [ ]:
query = "Make a course outline based on Gen AI course"
answer = qa_chain.run(query)
print(answer)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

4.50

30:00 Hours

14 Feb 2025

Free

Master Generative AI in Bangla

Boktiar Ahmed Bappy

Unlock the power of Generative AI with this comprehensive Bangla course! Dive into AI model training, deployment, and optimization while exploring popular tools like LLMs and GANs. Gain hands-on experience by building real-world projects, including chatbots, image generators, and creative content applications. Whether you're a beginner or an AI enthusiast, this course equips you with in-demand skills to excel in the evolving world of AI. Master the art of creating AI-driven text, images, and more, and become proficient in one of the most exciting fields of technology today!

4.95

100:00 Hours

16 Jun 2025

Free

Newest Courses

#Recently published courses

View All

Bestselling Courses

#Learn from bestselling courses

View All

Instr